In [1]:
import pandas as pd
import numpy as np

import requests
from requests import get
from bs4 import BeautifulSoup

from time import sleep
from random import randint

In [2]:
# Creating the lists we want to write into
titles = []
years = []
time = []
imdb_ratings = []
metascores = []
votes = []
us_gross = []

In [3]:
# Getting English translated titles from the movies
headers = {'Accept-Language': 'en-US, en;q=0.5'}


In [16]:
pages = np.arange(1, 1001, 50)
pages

array([  1,  51, 101, 151, 201, 251, 301, 351, 401, 451, 501, 551, 601,
       651, 701, 751, 801, 851, 901, 951])

In [19]:
 #Storing each of the urls of 50movies 
for page in pages:
    # Getting the contents from the each url
    page = requests.get('https://www.imdb.com/search/title/?groups=top_1000&start=' + str(page) + '&ref_=adv_nxt', headers=headers)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    # Aiming the part of the html we want to get the information from
    movie_div = soup.find_all('div', class_='lister-item mode-advanced')
    
    # Controling the loop’s rate by pausing the execution of the loop for a specified amount of time
    # Waiting time between requests for a number between 2-10 seconds
    sleep(randint(2,10))
    
    for container in movie_div:
        # Scraping the movie's name
        name = container.h3.a.text
        titles.append(name)
        
        # Scraping the movie's year
        year = container.h3.find('span', class_='lister-item-year').text
        years.append(year)
        
        # Scraping the movie's length
        runtime = container.find('span', class_='runtime').text if container.p.find('span', class_='runtime') else '-'
        time.append(runtime)
        
        # Scraping the rating
        imdb = float(container.strong.text)
        imdb_ratings.append(imdb)
        
        # Scraping the metascore
        m_score = container.find('span', class_='metascore').text if container.find('span', class_='metascore') else '-'
        metascores.append(m_score)
        
        # Scraping votes and gross earnings
        nv = container.find_all('span', attrs={'name':'nv'})
        vote = nv[0].text
        votes.append(vote)
        grosses = nv[1].text if len(nv) > 1 else '-'
        us_gross.append(grosses)

In [23]:
movies = pd.DataFrame({'movie':titles,
                       'year':years,
                       'time_minute':time,
                       'imdb_rating':imdb_ratings,
                       'metascore':metascores,
                       'vote':votes,
                       'gross_earning':us_gross})

movies.head()

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Dune,(2021),155 min,8.4,75,"47,719",-
1,Shang-Chi and the Legend of the Ten Rings,(2021),132 min,7.9,71,"91,430",-
2,The Matrix,(1999),136 min,8.7,73,"1,759,476",$171.48M
3,Shershaah,(2021),135 min,8.8,-,"102,150",-
4,Avengers: Endgame,(2019),181 min,8.4,78,"940,166",$858.37M


In [24]:
movies.dtypes

movie             object
year              object
time_minute       object
imdb_rating      float64
metascore         object
vote              object
gross_earning     object
dtype: object

In [25]:
# Cleaning 'year' column
movies['year'] = movies['year'].str.extract('(\d+)').astype(int)
movies.head(3)

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Dune,2021,155 min,8.4,75,"47,719",-
1,Shang-Chi and the Legend of the Ten Rings,2021,132 min,7.9,71,"91,430",-
2,The Matrix,1999,136 min,8.7,73,"1,759,476",$171.48M


In [26]:
# Cleaning 'time_minute' column
movies['time_minute'] = movies['time_minute'].str.extract('(\d+)').astype(int)
movies.head(3)

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Dune,2021,155,8.4,75,"47,719",-
1,Shang-Chi and the Legend of the Ten Rings,2021,132,7.9,71,"91,430",-
2,The Matrix,1999,136,8.7,73,"1,759,476",$171.48M


In [27]:
movies.dtypes

movie             object
year               int32
time_minute        int32
imdb_rating      float64
metascore         object
vote              object
gross_earning     object
dtype: object

In [28]:
movies

,movie,year,time_minute,imdb_rating,metascore,vote,gross_earning
0,Dune,2021,155,8.4,75,"47,719",-
1,Shang-Chi and the Legend of the Ten Rings,2021,132,7.9,71,"91,430",-
2,The Matrix,1999,136,8.7,73,"1,759,476",$171.48M
3,Shershaah,2021,135,8.8,-,"102,150",-
4,Avengers: Endgame,2019,181,8.4,78,"940,166",$858.37M
...,...,...,...,...,...,...,...
995,Kai Po Che,2013,120,7.8,40,"35,337",$1.12M
996,Vizontele,2001,110,8.0,-,"35,220",-
997,The Breath,2009,128,8.0,-,"32,800",-
998,Andaz Apna Apna,1994,160,8.1,-,"51,719",-
